<center>

# Compulsory Assignment 2 

## Natural Language Processing [KAN - CSCO1002U]
<center>

##  0. Importing Modules 

In [1]:
import pandas as pd 
import numpy as np
import matplotlib as plt
import seaborn as sns 
import matplotlib.pyplot as plt
import gensim.downloader as api
import nltk
import re
import gensim

from gensim.models import Word2Vec
from nltk.corpus import stopwords
from nltk import word_tokenize, WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from gensim.models import Word2Vec
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, ConfusionMatrixDisplay
from sklearn.ensemble import RandomForestClassifier

## 1. Importing the Data 

In [2]:
# Importing training and test set 
train = pd.read_csv("C:/Users/fredr/OneDrive/Documents/Master/Semester_2/NLP/Project/train.csv")
test = pd.read_csv("C:/Users/fredr/OneDrive/Documents/Master/Semester_2/NLP/Project/test.csv")
print(test.head())
print(train.head())

                                             article  topic  publication
0  intention presidentelect donald trump put end ...      7            1
1  board member uber arianna huffington active tr...      2            1
2  live blog european stock type live eikon news ...      7            1
3  peter bergen national security analyst vice pr...      7            3
4  m scott lutheran pastor one recent morning hap...      7            2
   publication                                            article  topic
0            2  fiat chrysler autombiles recalling jeep wrangl...      2
1            1  ottawa march china provided scientific evidenc...      2
2            1  cybersecurity company started two former natio...      2
3            2  washington federal aviation administration sai...      2
4            1  san francisco last october angellist company h...      2


In [3]:
print(train.info())
print(test.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 342876 entries, 0 to 342875
Data columns (total 3 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   publication  342876 non-null  int64 
 1   article      342876 non-null  object
 2   topic        342876 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 7.8+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41183 entries, 0 to 41182
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   article      41183 non-null  object
 1   topic        41183 non-null  int64 
 2   publication  41183 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 965.4+ KB
None


In [4]:
# Creating a sample 
train = train.sample(n=5000, random_state = 9)
test = test.sample(n=1000, random_state = 9)

In [5]:
# Shuffeling the data 
train = train.sample(frac=1).reset_index(drop=True)

## 2. Word Embeddings

### 2.1 Bag of Words 

In [6]:
# CountVectorizer 
BoW = CountVectorizer()
BoW.fit(train['article'])

CountVectorizer()

### 2.2 Word2Vect - Trained on our own data

Got codes from: 
https://spotintelligence.com/2023/02/15/word2vec-for-text-classification/

In [7]:
# Tokenizing the cleaned text
train_text = train['article']
train_tokens = train_text.apply(word_tokenize)

test_text = test['article']
test_tokens = test_text.apply(word_tokenize)

In [8]:
# Training the Word2Vec model 
wvT = Word2Vec(train_tokens, 
               vector_size = 100, 
               window = 5, 
               min_count = 2, 
               workers = 4)

In [9]:
# Function for vectorizing the text
def vectorize(tokens):
    words = tokens.split()
    words_vecs = [wvT.wv[word] for word in words if word in wvT.wv]
    if len(words_vecs) == 0:
        return np.zeros(100)
    words_vecs = np.array(words_vecs)
    return words_vecs.mean(axis=0)

### 2.3 Word2Vec - Trained by Google

Got codes from Exercise Class

In [10]:
# Google trained one 
wvG = api.load('word2vec-google-news-300')

In [11]:
# Function for computing the average vector 
# Gathered from Exercise 8

def avg_feature_vector(words, model, num_features):
    n_words = 0
    feature_vec = np.zeros((num_features, ), dtype='float32')
    for word in words:
        try:
            n_words += 1
            feature_vec = np.add(feature_vec, model[word])
        except KeyError:
            pass
    if n_words == 0:
        return feature_vec
    else:
        return feature_vec / n_words


## 3. Text Classification 

In [12]:
# Evaluation function that prints classification report and confusion matrix
def evaluate(y_test, y_pred, model_name):
    '''
    Function for Evaluating a model
    Prints out:
    --- Classification Report 
    --- Confusion Matrix
    '''
    
    print(f"\033[34m{model_name}\033[0m")
    
    #print Classification Report and Accuracy
    print(f" \033[32mClassification Report:\033[0m")
    print(classification_report(y_test, y_pred, zero_division=False))
    print(f" \033[32mAccuracy Score:\033[0m")
    print(accuracy_score(y_test, y_pred)*100)

### 3.1 Baseline Models

#### 3.1.1 Logistic Regression with Bag of Words

In [13]:
# Extracting features and content
y_test = test['publication']
y_train = train['publication']
X_test = test['article']
X_train = train['article']

In [14]:
# Vectorizer 
X_train_bow = BoW.transform(X_train)
X_test_bow = BoW.transform(X_test)

# Classifier 
lr_bow = LogisticRegression(max_iter=1000000)
lr_bow.fit(X_train_bow, y_train)

# Evaluate
yPred_lr_bow = lr_bow.predict(X_test_bow)
evaluate(y_test, yPred_lr_bow, 'Logistic Regression with BoW')

Logistic Regression with BoW
 Classification Report:
              precision    recall  f1-score   support

           1       0.82      0.77      0.79       335
           2       0.91      0.73      0.81       296
           3       0.76      0.68      0.72       254
           4       0.50      0.80      0.62        56
           5       0.71      1.00      0.83        39
           6       0.22      0.80      0.34        20

    accuracy                           0.75      1000
   macro avg       0.65      0.80      0.68      1000
weighted avg       0.80      0.75      0.76      1000

 Accuracy Score:
74.7


In [ ]:
confusion_matrix_bow = confusion_matrix(y_test, yPred_lr_bow)
classes_bow = np.unique(y_test)
cm_display = ConfusionMatrixDisplay(confusion_matrix = confusion_matrix_bow, display_labels = classes_bow)
plt.show()

#### 3.1.2 Random Forest with Bag of Words

In [15]:
# Classifier 
rf_bow = RandomForestClassifier()
rf_bow.fit(X_train_bow, y_train)

# Evaluate
yPred_rf_bow = rf_bow.predict(X_test_bow)
evaluate(y_test, yPred_rf_bow, 'Random Forest with BoW')

Random Forest with BoW
 Classification Report:
              precision    recall  f1-score   support

           1       0.80      0.81      0.80       335
           2       0.84      0.70      0.76       296
           3       0.72      0.69      0.70       254
           4       0.79      0.80      0.80        56
           5       0.88      0.97      0.93        39
           6       0.23      0.80      0.35        20

    accuracy                           0.75      1000
   macro avg       0.71      0.80      0.72      1000
weighted avg       0.78      0.75      0.76      1000

 Accuracy Score:
75.1


#### 3.1.3 Logistic Regression with own trained Word2Vec embeddings

In [16]:
# Vectorizing
X_train_wvT = np.array([vectorize(train_tokens) for train_tokens in X_train])
X_test_wvT = np.array([vectorize(train_tokens) for train_tokens in X_test])

In [17]:
# Classifier 
lr_wvT = LogisticRegression(max_iter=1000000)
lr_wvT.fit(X_train_wvT, y_train)

# Evaluate
yPred_lr_wvT = lr_wvT.predict(X_test_wvT)
evaluate(y_test, yPred_lr_wvT, 'Logisitic Regression with own W2V')

Logisitic Regression with own W2V
 Classification Report:
              precision    recall  f1-score   support

           1       0.82      0.67      0.74       335
           2       0.84      0.55      0.66       296
           3       0.59      0.60      0.60       254
           4       0.26      0.55      0.35        56
           5       0.47      0.97      0.64        39
           6       0.16      0.60      0.26        20

    accuracy                           0.62      1000
   macro avg       0.52      0.66      0.54      1000
weighted avg       0.71      0.62      0.65      1000

 Accuracy Score:
62.1


In [ ]:
confusion_matrix_wvt = confusion_matrix(y_test, yPred_lr_wvT)
classes_bow = np.unique(y_test)
cm_display = ConfusionMatrixDisplay(confusion_matrix = confusion_matrix_wvt, display_labels = classes_bow)
plt.show()

#### 3.1.4 Random Forest with own trained Word2Vec embeddings

In [18]:
# Classifier 
rf_wvT = RandomForestClassifier()
rf_wvT.fit(X_train_wvT, y_train)

# Evaluate
yPred_rf_wvT = rf_wvT.predict(X_test_wvT)
evaluate(y_test, yPred_rf_wvT, 'Random Forest with own W2V')

Random Forest with own W2V
 Classification Report:
              precision    recall  f1-score   support

           1       0.82      0.67      0.74       335
           2       0.76      0.43      0.55       296
           3       0.63      0.66      0.64       254
           4       0.24      0.48      0.32        56
           5       0.36      0.90      0.51        39
           6       0.09      0.40      0.15        20

    accuracy                           0.59      1000
   macro avg       0.48      0.59      0.48      1000
weighted avg       0.69      0.59      0.61      1000

 Accuracy Score:
58.699999999999996


#### 3.1.3 Logistic Regression with Google trained Word2Vec embeddings

In [19]:
# Constructing average vectors 
train_features = []
vectorsize = 300
for t in train_tokens:
    train_features.append(avg_feature_vector(t, wvG, vectorsize))    


In [20]:
test_features = []
vectorsize = 300
for t in test_tokens:
    test_features.append(avg_feature_vector(t, wvG, vectorsize))    

In [21]:
# Splitting the data 
X_train_wvG = train_features
X_test_wvG = test_features
y_train_wvG = train['publication']
y_test_wvG = test['publication']

In [22]:
# Classifier  
lr_wvG = LogisticRegression(max_iter=1000000)
lr_wvG.fit(X_train_wvG, y_train_wvG)

# Evaluate
yPred_lr_wvG = lr_wvG.predict(X_test_wvG)
evaluate(y_test_wvG, yPred_lr_wvG, 'Logistic Regression with Google W2V')

Logistic Regression with Google W2V
 Classification Report:
              precision    recall  f1-score   support

           1       0.81      0.64      0.72       335
           2       0.83      0.40      0.54       296
           3       0.61      0.61      0.61       254
           4       0.26      0.52      0.34        56
           5       0.31      0.95      0.47        39
           6       0.12      0.60      0.19        20

    accuracy                           0.57      1000
   macro avg       0.49      0.62      0.48      1000
weighted avg       0.70      0.57      0.60      1000

 Accuracy Score:
56.699999999999996


#### 3.1.5 Random Forest with Google trained Word2Vec embeddings

In [26]:
# Classifier 
rf_wvG = RandomForestClassifier()
rf_wvG.fit(X_train_wvG, y_train_wvG)

# Evaluate 
yPred_rf_wvG = rf_wvG.predict(X_test_wvG)
evaluate(y_test_wvG, yPred_rf_wvG, 'Random Forest with Google W2V')

Random Forest with Google W2V
 Classification Report:
              precision    recall  f1-score   support

           1       0.80      0.63      0.70       335
           2       0.69      0.42      0.52       296
           3       0.60      0.59      0.59       254
           4       0.24      0.52      0.33        56
           5       0.30      0.82      0.44        39
           6       0.10      0.45      0.17        20

    accuracy                           0.55      1000
   macro avg       0.46      0.57      0.46      1000
weighted avg       0.65      0.55      0.58      1000

 Accuracy Score:
55.2
